In [1]:
# imports
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score,log_loss,r2_score
import warnings
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer,make_column_selector,make_column_transformer
from sklearn.svm import SVC
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [2]:
sonar_df = pd.read_csv(r'..\Cases\Sonar\Sonar.csv')
sonar_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V52,V53,V54,V55,V56,V57,V58,V59,V60,Class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [3]:
X= sonar_df.drop(columns=['Class'])
y= sonar_df['Class']
# le= LabelEncoder()
# y= le.fit_transform(y)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=25,stratify=y)

In [4]:
svm1=SVC(kernel='linear',probability=True,random_state=25)
svm2=SVC(kernel='rbf',probability=True,random_state=25)
knn=KNeighborsClassifier()
voting=VotingClassifier([('svm1',svm1),('svm2',svm2),('knn',knn)])

In [5]:
voting.fit(X_train,y_train)
y_pred=voting.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.7619047619047619


## Soft voting

We take the average of the probabilities given by the model and then do theshold on this value

In [6]:
svm1=SVC(kernel='linear',probability=True,random_state=25)
svm2=SVC(kernel='rbf',probability=True,random_state=25)
knn=KNeighborsClassifier()
soft_voting=VotingClassifier([('svm1',svm1),('svm2',svm2),('knn',knn)],voting='soft')

In [7]:
soft_voting.fit(X_train,y_train)
y_pred=soft_voting.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.7777777777777778


In [8]:
y_pred_proba=soft_voting.predict_proba(X_test)
roc_auc_score(y_test,y_pred_proba[:,1])

0.896551724137931

Evaluting idividual model performance


In [9]:
voting.named_estimators_

{'svm1': SVC(kernel='linear', probability=True, random_state=25),
 'svm2': SVC(probability=True, random_state=25),
 'knn': KNeighborsClassifier()}

In [10]:
#voting.named_estimators['knn'].fit(X_train,y_train)
y_pred_proba=voting.named_estimators_['knn'].predict_proba(X_test)
roc_auc_score(y_test,y_pred_proba[:,1])

0.8676470588235294

In [11]:
#voting.named_estimators['svm1'].fit(X_train,y_train)
y_pred_proba=voting.named_estimators_['svm1'].predict_proba(X_test)
roc_auc_score(y_test,y_pred_proba[:,1])

0.8052738336713996

In [12]:
#voting.named_estimators['svm2'].fit(X_train,y_train)
y_pred_proba=voting.named_estimators_['svm2'].predict_proba(X_test)
roc_auc_score(y_test,y_pred_proba[:,1])

0.8894523326572008